In [65]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import  mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('energydata_complete.csv')

In [4]:
df.shape

(19735, 29)

In [6]:
#ratio of null values
df.isnull().sum()/df.shape[0] *100

date           0.0
Appliances     0.0
lights         0.0
T1             0.0
RH_1           0.0
T2             0.0
RH_2           0.0
T3             0.0
RH_3           0.0
T4             0.0
RH_4           0.0
T5             0.0
RH_5           0.0
T6             0.0
RH_6           0.0
T7             0.0
RH_7           0.0
T8             0.0
RH_8           0.0
T9             0.0
RH_9           0.0
T_out          0.0
Press_mm_hg    0.0
RH_out         0.0
Windspeed      0.0
Visibility     0.0
Tdewpoint      0.0
rv1            0.0
rv2            0.0
dtype: float64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         19735 non-null  object 
 1   Appliances   19735 non-null  int64  
 2   lights       19735 non-null  int64  
 3   T1           19735 non-null  float64
 4   RH_1         19735 non-null  float64
 5   T2           19735 non-null  float64
 6   RH_2         19735 non-null  float64
 7   T3           19735 non-null  float64
 8   RH_3         19735 non-null  float64
 9   T4           19735 non-null  float64
 10  RH_4         19735 non-null  float64
 11  T5           19735 non-null  float64
 12  RH_5         19735 non-null  float64
 13  T6           19735 non-null  float64
 14  RH_6         19735 non-null  float64
 15  T7           19735 non-null  float64
 16  RH_7         19735 non-null  float64
 17  T8           19735 non-null  float64
 18  RH_8         19735 non-null  float64
 19  T9  

In [8]:
#categorical features
categorical = df.select_dtypes(include =[np.object])
print("Categorical Features in Train Set:",categorical.shape[1])

#numerical features
numerical= df.select_dtypes(include =[np.float64,np.int64])
print("Numerical Features in Train Set:",numerical.shape[1])

Categorical Features in Train Set: 1
Numerical Features in Train Set: 28


In [9]:
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [38]:
df = df.drop(['date', 'lights'], axis = 1) 
df.head()

,Appliances,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,60,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,60,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,50,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,50,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,60,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [40]:
scaler = MinMaxScaler()

energydata_scaled = pd.DataFrame(scaler.fit_transform(df),  columns = df.columns)

#get features and labels
X = energydata_scaled.drop(columns=['Appliances'])

y = energydata_scaled['Appliances']

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print('X_train shape: {}'.format(X_train.shape))
print('y_train shape: {}'.format(y_train.shape))
print('X_test shape: {}'.format(X_test.shape))
print('y_test shape: {}'.format(y_test.shape))

X_train shape: (13814, 26)
y_train shape: (13814,)
X_test shape: (5921, 26)
y_test shape: (5921,)


In [42]:
reg_df = energydata_scaled[['T2', 'T6']]
reg_df.head()

,T2,T6
0,0.225345,0.381070
1,0.225345,0.375443
2,0.225345,0.367487
3,0.225345,0.363800
4,0.225345,0.361859


In [43]:
x= reg_df['T2'].values.reshape(-1,1)
y = reg_df['T6'].values.reshape(-1,1)

In [44]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3, random_state=42)

In [45]:
lin_regr = linear_model.LinearRegression()
lin_regr.fit(xtrain, ytrain)
pred = lin_regr.predict(xtest)

In [46]:
r2_score = r2_score(ytest, pred)
print('R-squared:',(round(r2_score, 2)))

R-squared: 0.64


In [49]:
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)
energy_pred = regr.predict(X_test)
print("Training set score: {:.3f}".format(regr.score(X_train, y_train)))
print("Test set score: {:.3f}".format(regr.score(X_test, y_test)))

Training set score: 0.145
Test set score: 0.149


In [52]:
mae = mean_absolute_error(y_test, energy_pred)
print('MAE:',(round(mae, 2)))
rss = np.sum(np.square(y_test - energy_pred))
print('RSS:',(round(rss, 2)))
rmse = np.sqrt(mean_squared_error(y_test, energy_pred))
print('RMSE:',(round(rmse, 3)))
r2_score = r2_score(y_test, energy_pred)
print('R-squared:',(round(r2_score, 2)))

MAE: 0.05
RSS: 45.34
RMSE: 0.088
R-squared: 0.15


In [53]:
def get_weights_df(model, feat, col_name):
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features', col_name]
    weights_df[col_name].round(3)
    
    return weights_df

In [54]:
linear_model_weights = get_weights_df(regr, X_train, 'Linear_Model_Weight')
linear_model_weights

,Features,Linear_Model_Weight
0,rv2,-2.219382e+11
1,RH_2,-4.567672e-01
2,T_out,-3.219552e-01
3,T2,-2.361994e-01
4,T9,-1.899314e-01
5,RH_8,-1.576020e-01
6,RH_out,-7.768452e-02
7,RH_7,-4.455266e-02
8,RH_9,-3.980932e-02
9,T5,-1.566960e-02


In [62]:
ridge_reg = Ridge(alpha=0.4)
ridge_reg.fit(X_train, y_train)
ridge_pred = ridge_reg.predict(X_test)
print("Training set score: {:.3f}".format(ridge_reg.score(X_train, y_train)))
print("Test set score: {:.3f}".format(ridge_reg.score(X_test, y_test)))

Training set score: 0.145
Test set score: 0.149


In [63]:
rmse = np.sqrt(mean_squared_error(y_test, ridge_pred))
print('RMSE:',(round(rmse, 3)))

RMSE: 0.088


In [64]:
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(X_train, y_train)
lasso_pred = lasso_reg.predict(X_test)
print("Training set score: {:.3f}".format(lasso_reg.score(X_train, y_train)))
print("Test set score: {:.3f}".format(lasso_reg.score(X_test, y_test)))

Training set score: 0.025
Test set score: 0.027


In [66]:
lasso_weights_df = get_weights_df(lasso_reg, X_train, 'Lasso_weight')
lasso_weights_df

,Features,Lasso_weight
0,RH_out,-0.049557
1,RH_8,-0.000110
2,T1,0.000000
3,Tdewpoint,0.000000
4,Visibility,0.000000
5,Press_mm_hg,-0.000000
6,T_out,0.000000
7,RH_9,-0.000000
8,T9,-0.000000
9,T8,0.000000


In [67]:
rmse = np.sqrt(mean_squared_error(y_test, lasso_pred))
print('RMSE:',(round(rmse, 3)))

RMSE: 0.094
